In [ ]:
#Import necessary packages
import requests
from bs4 import BeautifulSoup
import pandas as pd



In [ ]:
#make the python's request look like it is from a real computer (supposedly...)
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)",
    "Accept-Language": "en-US,en;q=0.9"
}
#this url for 2025 World Open
url = 'https://cuetracker.net/tournaments/world-open/2025/6445'
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
#response 200 means successful!
print(response)


'Final'

In [ ]:
#The following scrapes all matches using the keyword "match" since they are split into "match row odd" and "match row even"

#Create a pandas dataframe with the following column names.
matches_df = pd.DataFrame(columns = ['Player1', 'Player2', 'Score1','Score2','Best Of', 'Round Name', 'Tournament Name'])

#All matches are stroed under div, class name "match row even" or "match row odd".
#In any case .find_all("div", class_ = "match") will find all class names containing the string "match"
all_matches = soup.find_all("div", class_ = "match")

#Get the tournament name.
tournament_name = soup.find("h1", class_ = "text-center")
#Get rid of all invisible characters
#A small thing to note is that .find_all returns a list and doesn't have .text method, so we have to loop through it to use 
#.text method
tournament_name_str = [tournament_name.text.strip()]
#Couter for writing row data into dataframe row-by-row.
count = 0

#.find_all returns a list, we are looping through this list
for match in all_matches:
    #Initialize empty list, which is going to hold data from each match
    match_data = []

    #Players names are under this class name, I took a portion of the class name because it is common for both players.
    #You can F12 check the website to see specific names
    players_names = match.find_all("div", class_ = "matchResultText")

    for player_name in players_names:
        # Turn the string into a list, so .extend works properly, o.w. it extends character-by-character
        player = [player_name.text.strip()]
        match_data.extend(player)

    #Same thing here, the actual scores are under 'span'
    players_scores = match.find_all("span", class_ = "matchResultText")

    for scores in players_scores:
        score = [scores.text.strip()]
        match_data.extend(score)

    #The parentheses () is interpreted as negative sign in excel, so just get rid of "()"
    best_of = [match.find("span", class_ = "best_of").text.strip("()")]
    match_data.extend(best_of)

    #This for getting the stage, e.g., Final, Semi-Final, etc.
    round_name = [match.find("h5").text.strip()]
    match_data.extend(round_name)

    
    # I don't quite know how to handle non-played(walkover) matches as they don't have a "Played On" date
    # Obviously the following doesn't work somehow
    # match_date = match.find("div", class_ = "col-12 played_on")
    # if match_date is None:
    #     match_date_str = []
    # else:
    #     match_date_str = [match_date.text.strip()]

    # match_data.extend(match_date_str)

    #Put the tournament name in the last column
    match_data.extend(tournament_name_str)
    
    #Write this data into one row in the dataframe
    matches_df.loc[count] = match_data
    count+=1

#You need to change the file path to your actual path.
matches_df.to_csv(f'/Users/huangpubo/Documents/Erdos/EloRatingProject/{tournament_name_str[0]}-all-matches.csv')
    

In [261]:
ls = []
bestof = [match.find("span", class_ = "best_of").text]
ls.extend(bestof)
print (ls)

['(35)']


In [235]:
url = 'https://cuetracker.net/tournaments/2024'
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)",
    "Accept-Language": "en-US,en;q=0.9"
}
response = requests.get(url, headers=headers)
print(response)
soup = BeautifulSoup(response.text, 'html.parser')


<Response [200]>


In [ ]:
#this for Matches Won - Year 2025 under the Statistics tab
url = 'https://cuetracker.net/statistics/matches-and-frames/won/year/2025'
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)",
    "Accept-Language": "en-US,en;q=0.9"
}
response = requests.get(url, headers=headers)
print(response)
soup = BeautifulSoup(response.text, 'html.parser')


<Response [200]>


In [31]:
#initialize empty dataframe
players_stat = pd.DataFrame(columns=['Name', 'Tournaments Played', 'Matches Played', 'Matches Won', 'Match-win percentage'])
matches_won_for_players = soup.find_all("tr")
title = soup.find("h1").text.strip()
current_row_num = 0
for player in matches_won_for_players:

    #all data are stored under td tags and in the order name, tournaments played, matches played, matches won, and match-win percentage
    row_data = player.find_all("td")

    texts = [data.text.strip() for data in row_data]
    texts = texts[1::] #removes the first empty string
    
    players_stat.loc[current_row_num] = texts

    current_row_num+=1

players_stat.to_csv(f'/Users/huangpubo/Documents/Erdos/EloRatingProject/{title}.csv')
